In [150]:
import numpy as np
import skimage
import pandas as pd
import anndata as ad
from collections import namedtuple
import networkx as nx
import math
from shapely.geometry import Point
import sys
import getopt
import anndata as ad
import pickle
from pathlib import Path
from scipy.spatial import Voronoi, voronoi_plot_2d
from shapely.geometry import Polygon
import string
import random
import geopandas

In [151]:
class snakemake():
    
    input = ['/gpfs/commons/groups/nygcfaculty/PySeq/spatial_analysis/m1a.h5ad','/gpfs/commons/groups/nygcfaculty/PySeq/spatial_analysis/m1a.csv']
    output = ['/gpfs/commons/home/jsingh/results/neighbor_dict.pkl','/gpfs/commons/home/jsingh/results/cell_type_per_node.pkl']

In [152]:
adata = ad.read(snakemake.input[0])

/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [153]:
x_coord = adata.X[:,-2]
y_coord = adata.X[:,-1]

points = []
for i,y in zip(x_coord,y_coord):
    points.append([i,y])

In [154]:
mat = np.column_stack((x_coord, y_coord))

spatial_point_list = []
for s in mat:
    spatial_point_list.append(Point(s))

In [155]:
indexes = np.random.choice(np.shape(points)[0],round(np.shape(points)[0]*0.25), replace=False)

In [156]:
down_sampled_points = []
for i in indexes:
    down_sampled_points.append(points[i])

In [157]:
x_coord = adata.X[:,-2]
y_coord = adata.X[:,-1]

points = []

for i,y in zip(x_coord,y_coord):
    points.append([i,y])

vor = Voronoi(down_sampled_points)

In [158]:
indices_list = []
for region in vor.regions:
    indices_list.append(region)

In [159]:
polygon_list = []
for indice in indices_list:
    polygon_vertex_list = []
    for i in range(len(indice)):
        polygon_vertex_list.append(vor.vertices[indice[i]])
    polygon_list.append(polygon_vertex_list)

In [160]:
polygons = []
for pol in polygon_list:
    polygons.append(Polygon(pol))

In [161]:
def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

In [162]:
polygon_id = {}
for pol in polygons:
    polygon_id.update({id_generator():pol})

In [163]:
pol_point_dict = {}
for pol,ids in zip(polygons,polygon_id.keys()):
    pol_point_dict.update({ids:[]})
    

In [164]:
for ids in polygon_id.keys():
    for p in spatial_point_list:
        if polygon_id[ids].contains(p) == True:
            pol_point_dict[ids].append(p)
            
    

In [165]:
df = pd.read_csv(snakemake.input[1])

In [167]:
pol_cell_type = {}
for key in pol_point_dict.keys():
    pol_cell_type.update({key:[]})


cell_type = {}
for i,ty in zip(df.id,df.type):
    cell_type.update({i: ty})
 
for key in pol_cell_type.keys():
    for p in pol_point_dict[key]:
        val = np.asarray([p.x, p.y])
        counter = counter + 1
        idx = np.where(mat == val)[0][1]
        pol_cell_type[key].append(cell_type[idx+1])
        
       
        
    
    


In [168]:
counter

180

In [134]:
type_series = pd.Series(df.type, dtype = "category")
cell_type_list = type_series.cat.categories.to_list()
gdf = geopandas.GeoDataFrame(geometry = polygons, columns = cell_type_list, index = pol_cell_type.keys())
gdf = gdf.fillna(0)

In [145]:
counter = 0
for key in pol_cell_type.keys():
    fill_df = pd.Series(pol_cell_type[key]).value_counts()
    for i in fill_df.index:
        gdf.at[key,i] = fill_df[i]
        counter = counter + 1
        

/tmp/ipykernel_27232/984467838.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  fill_df = pd.Series(pol_cell_type[key]).value_counts()


In [147]:
gdf

,Astrocytes,Cholinergic neurons,Endothelial cells,Excitatory neurons,Inhibitory neurons,Microglia/Macrophages,Oligodendrocytes,geometry
XUNCMB,2,0,0,0,0,0,2,"POLYGON ((1399.000 1402.024, 861.333 1405.620,..."
QXLBYZ,2,0,1,0,1,0,0,"POLYGON ((7292.880 -4604.804, 1364.155 1353.80..."
2HIG1N,3,0,2,0,2,0,1,"POLYGON ((861.333 1405.620, 1399.000 1402.024,..."
Q3K4FH,0,0,0,0,0,0,0,GEOMETRYCOLLECTION EMPTY
T2I18O,2,0,0,0,0,0,1,"POLYGON ((885.602 795.892, 1373.065 652.356, 1..."
1JIWSZ,3,0,0,0,0,0,1,"POLYGON ((635.975 755.620, 861.333 1405.620, 7..."
XYZ4S3,4,0,1,0,0,0,2,"POLYGON ((879.107 1404.540, 987.227 1299.713, ..."
YFHY02,1,0,0,0,0,0,1,"POLYGON ((608.731 756.402, 181.281 1252.671, 1..."
28YODQ,0,1,1,3,1,0,0,"POLYGON ((453.254 723.017, -3200.730 -249.578,..."
UJM4ZX,1,1,0,1,0,0,0,"POLYGON ((608.731 756.402, 634.439 756.381, 59..."
